<a href="https://colab.research.google.com/github/AhmadKElsayed/Chilling/blob/main/FAISS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install faiss-cpu
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

# Example:

In [2]:
import os
import json
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

In [3]:
docs = [
    "This is document 1.",
    "Another example text.",
    "AI-powered systems are fun.",
    "FAISS is a vector search library.",
    "Retrieval-Augmented Generation rocks."
]


In [4]:
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(docs)

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)

# Add embeddings to index
index.add(embeddings.astype('float32'))

# Save a mapping from FAISS index to original documents
index_to_doc = {i: doc for i, doc in enumerate(docs)}

#Saving json
with open("index_metadata.json", "w") as f:
  json.dump(index_to_doc, f)

# Saving FAISS DB
faiss.write_index(index, "my_vector_index.faiss")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
query_embedding = model.encode(["What is FAISS?"]).astype('float32')
D, I = index.search(query_embedding, k=3)

print(I)

for idx in I[0]:
    print(index_to_doc[idx])

[[3 1 0]]
FAISS is a vector search library.
Another example text.
This is document 1.


In [6]:
#Loading FAISS
loaded_index = faiss.read_index("my_vector_index.faiss")
with open("index_metadata.json", "r") as f:
    index_to_metadata = json.load(f)

print(loaded_index.ntotal)

5


In [7]:
# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Load existing index
index = faiss.read_index("my_vector_index.faiss")

# Load existing metadata
with open("index_metadata.json", "r") as f:
    index_to_doc = json.load(f)

# Determine next available index ID
next_id = max(map(int, index_to_doc.keys())) + 1 if index_to_doc else 0

# New docs to add
new_docs = ["KAMEL IS THE BEST LLM Engineer", "I LOVE MOZA", "KIMA IS OVERRATED.. BOOOOOOOO"]

# Generate embeddings for new docs
new_embeddings = model.encode(new_docs).astype('float32')

# Add new embeddings to FAISS index
index.add(new_embeddings)

# Update metadata mapping
for i, doc in enumerate(new_docs):
    index_to_doc[next_id + i] = doc

# Save updated index
faiss.write_index(index, "my_vector_index.faiss")

# Save updated metadata
with open("index_metadata.json", "w") as f:
    json.dump(index_to_doc, f)


In [8]:
#Check if the files were added
loaded_index = faiss.read_index("my_vector_index.faiss")
with open("index_metadata.json", "r") as f:
    index_to_metadata = json.load(f)

print(loaded_index.ntotal)

8


# For Files:

In [ ]:
# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Initialize FAISS index
dimension = 384  # MiniLM has 384 dimensions
index = faiss.IndexFlatL2(dimension)

# Metadata mapping
index_to_metadata = {}

# Loop through files
data_dir = "data"
for i, filename in enumerate(os.listdir(data_dir)):
    if filename.endswith(".txt"):
        path = os.path.join(data_dir, filename)
        with open(path, 'r') as f:
            content = f.read()

        # Create embedding
        embedding = model.encode(content).astype('float32')
        index.add(np.array([embedding]))

        # Save filename and file path
        index_to_metadata[i] = {"title": filename, "path": path}


#Saving FAISS Object and Metadata dictionary
faiss.write_index(index, "my_vector_index.faiss")

with open("index_metadata.json", "w") as f:
  json.dump(index_to_metadata, f)

In [ ]:
query = "What is FAISS?"
query_embedding = model.encode([query]).astype('float32')

D, I = index.search(query_embedding, k=2)

for idx in I[0]:
    meta = index_to_metadata[idx]
    print(f"Title: {meta['title']}")
    print(f"Content: {meta['path']}\n")

In [ ]:
# Loading the FAISS and metadata
index = faiss.read_index("my_vector_index.faiss")

with open("index_metadata.json", "r") as f:
    index_to_metadata = json.load(f)

# Next index
next_id = max(map(int, index_to_metadata.keys())) + 1 if index_to_metadata else 0

# New files additon
data_dir = "data"
existing_files = {meta["title"] for meta in index_to_metadata.values()}

for filename in os.listdir(data_dir):
    if filename.endswith(".txt") and filename not in existing_files:
        path = os.path.join(data_dir, filename)
        with open(path, 'r') as f:
            content = f.read()

        embedding = model.encode(content).astype('float32')
        index.add(np.array([embedding]))

        # Save metadata
        index_to_metadata[next_id] = {"title": filename, "path": path}
        next_id += 1

# Saving the new FAISS and json
faiss.write_index(index, "my_vector_index.faiss")

with open("index_metadata.json", "w") as f:
    json.dump(index_to_metadata, f)